In [ ]:
from pyspark.sql import SparkSession


In [ ]:
spark = SparkSession.builder\
.appName("Olist Ecommerce Performance Optimization")\
.config("spark.executor.memory",'6g')\
.config("spark.executor.cores",'4')\
.config("spark.executor.instances",'2')\
.config("spark.driver.memory",'4g')\
.config("spark.driver.maxResultSize",'2g')\
.config("spark.sql.shuffle.partitions",'64')\
.config('spark.default.parallelism','64')\
.config("spark.sql.adaptive.enabled",'true')\
.config("spark.sql.adaptive.coalescePartition.enabled",'true')\
.config("spark.sql.files.maxPartitionsBytes",'64MB')\
.config('spark.sql.files.openCostInBytes','2MB')\
.config("spark.memory.fraction",0.8)\
.config("spark.memory.storageFraction",0.8)\
.config('spark.sql.autoBroadcastJoinThreshold',20*1024*1024)\
.getOrCreate()

In [7]:
full_orders_df = spark.read.format('parquet').load("/content/full_orders_dataset")

#Optimized Join Strategies

##Broadcast

In [ ]:
customers_broadcast_df = broadcast(customers_df)
optimized_broadcast_join = full_orders_df.join(customers_broadcast_df,'customer_id')

##Sort and Merge Join

In [ ]:
sorted_customers_df = customers_df.sortWithinPartitions('customer_id')
sorted_orders_df = full_orders_df.sortWithinPartitions('customer_id')

optimizes_merge_full_orders_df = sorted_orders_df.join(sorted_customers_df,'customer_id')


##Bucket Join

In [ ]:
bucketed_customers_df = customers_df.sortWithinPartitions('customer_id')
bucketed_orders_df = full_orders_df.sortWithinPartitions('customer_id')

bucket_join_df= bucketed_orders_df.join(bucketed_customers_df,'customer_id')


#Skew Join handling

In [ ]:
skwe_handles_join = full_orders_df.join(customers_df,'customer_df')